In [4]:
import json
import pandas as pd
from mm_analytics.utilities import NpEncoder
DATA_ROOT = "/Users/andrewgrowney/data/kaggle/marchmadness-2024"

In [74]:
seasons_df = pd.read_csv(f"{DATA_ROOT}/MSeasons.csv") # Season, DayZero, RegionW, RegionX, RegionY, RegionZ
seeds_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneySeeds.csv") # Season, Seed, TeamID
slots_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneySlots.csv") # Season, Slot[R1W4], StrongSeed[W04], WeakSeed[W13]
seed_round_slots_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneySeedRoundSlots.csv") # Seed, GameRound[1], GameSlot[R1W4], EarlyDayNum, LateDayNum
results_df = pd.read_csv(f"{DATA_ROOT}/MNCAATourneyCompactResults.csv") # Season, DayNum, WTeamID, LTeamID, WScore, LScore, WLoc, NumOT

In [73]:
slots_df

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12
...,...,...,...,...
2447,2023,R6CH,R5WX,R5YZ
2448,2023,W16,W16a,W16b
2449,2023,X16,X16a,X16b
2450,2023,Y11,Y11a,Y11b


In [100]:
TEAM_DATA_ROOT = "../data/web/ts"
def get_team_data(tid, season, seed_data = None):
    with open(f"{TEAM_DATA_ROOT}/{tid}_{season}.json") as f:
        team_data = json.load(f)
    team_seed = team_data.get("tournament", {}).get("seed", None) if seed_data is None else seed_data[tid]
    team_ranks = {
            "AdjOE": team_data["stat_rankings"]["AdjOE"],
            "AdjDE": team_data["stat_rankings"]["AdjDE"],
            "AdjNE": team_data["stat_rankings"]["AdjNE"],
            "SOS": team_data["stat_rankings"]["SOS"],
            "Poss": team_data["stat_rankings"]["Poss"]
        }
    return {
        "Seed": team_seed,
        "Rec": team_data["record"]["overall"],
        "Ranks": team_ranks,
        "Sims": [{"i": i["id"], "y": i["year"], "s": i["avg"], "e": i["er"]} for i in team_data["similar_teams"]]
    }

In [ ]:
"""Pseudocode for getting the probabilities
from the target_slot, find the slots that feed into it -> feed_slots

for each feed_slot:
    if there is a record in slot_winners for the slot:
        return that 
    else:

Step 1: Build out tree of slots
ex: { 
    "W11a": {"day": 134, "prob": None, "feed_slots": ["W11a"]},
    "W11b": {"day": 134, "prob": None, "feed_slots": ["W11b"]},
    "W1":   {"day": 136, "prob": None, "feed_slots": ["W1"]},
    "W11": {"day": 136, "prob": None, "feed_slots":  ["W11a", "W11b"]},
    "R1W1": {"day": 1, "prob": None, "feed_slots": ["W1", "W16"], "winner": None },
    "R1W2": {"day": 1, "prob": None, "feed_slots": ["W8", "W9"], "winner": None },
    "R2W1": {"day": 2, "prob": None, "feed_slots": ["R1W1", "R1W2"], "winner": None }, 
}
Step 2: Order the slots by day
{ 
    "W1":   {"day": 0, "prob": {1242: 1.0}, "feed_slots": ["W1"]},
    "W16":   {"day": 0, "prob": {1113: 1.0}, "feed_slots": ["W16"]},
    "W11a":  {"day": 0, "prob": {1113: 1.0}, "feed_slots": ["W11a"]},
    "W11a":  {"day": 0, "prob": {1113: 1.0}, "feed_slots": ["W11a"]},
    "R1W1": {"day": 1, "prob": None, "feed_slots": ["W1", "W16"], "winner": None },
    "R1W2": {"day": 1, "prob": None, "feed_slots": ["W8", "W9"], "winner": None },
    "R2W1": {"day": 2, "prob": None, "feed_slots": ["R1W1", "R1W2"], "winner": None }, 
}
Step 3: Fill out the prob for each slot
{
    "W1":   {"day": 0, "prob": {1242: 1.0}, "feed_slots": ["W1"]},
    "W16":  {"day": 0, "prob": {1113: 1.0}, "feed_slots": ["W16"]},
    "R1W1": {"day": 1, "prob": {1242: 0.5, 1113: 0.5}, "feed_slots": ["W1", "W16"] },
}
for slot_id, slot_val in slots.items():
    if len(slot_val["feed_slots"]) == 1:
        slot_val["prob"] = { team_seeds[slot_val["feed_slots"][0]]: 1.0 }
    else:
        slot_prob = {}
        feed_slot_probs = [slots[fs]["prob"] for fs in slot_val["feed_slots"]]
        for tid, feed_prob in feed_slot_probs[0].items():
            total_t_prob = 0
            for opp, opp_prob in feed_slot_probs[1].items():
                total_t_prob += feed_prob * opp_prob * get_win_prob(model_id, tid, opp)[0]
            slot_prob[tid] = total_t_prob
        for tid, feed_prob in feed_slot_probs[1].items():
            total_t_prob = 0
            for opp, opp_prob in feed_slot_probs[0].items():
                total_t_prob += feed_prob * opp_prob * get_win_prob(model_id, tid, opp)[0]
            slot_prob[tid] = total_t_prob
        slot_val["prob"] = slot_prob

"""

# def get_win_prob(model_id, team1, team2):
#     """Get the probability of team1 winning against team2
#     """
#     return [0.5, 0.5]

# def get_slot_probabilities(seed_teams):
#     """calculate the chances of each team winning the slot
#     """
#     slots = {}
#     # Initialize seeds as slots
#     for sid, tid in seed_teams.items():
#         slots[sid] = { "day": 136, "feed_slots": [sid] }
#     # Assign Feed Slots
#     for _, row in slots_df.iterrows():
#         slots[row.Slot] = { "feed_slots": [row.StrongSeed, row.WeakSeed] } # Slot[R1W4], StrongSeed[W04], WeakSeed[W13]
    
#     for _, row in seed_round_slots_df.iterrows(): # Seed, GameRound[1], GameSlot[R1W4], EarlyDayNum, LateDayNum
#         slot_id = row.GameSlot
#         slots[slot_id]["day"] = row.EarlyDayNum
#     print({k: v["day"] for k, v in slots.items()})
#     # Order slots by round
#     slots = {k: v for k, v in sorted(slots.items(), key=lambda item: item[1]["day"])}
#     print({k: v["day"] for k, v in slots.items()})
#     # Fill out the prob for each slot
#     for slot_id, slot_val in slots.items():
#         if len(slot_val["feed_slots"]) == 1:
#             print(f"Filled {slot_id} with {seed_teams[slot_val['feed_slots'][0]]}")
#             slots[slot_id]["prob"] = { seed_teams[slot_val["feed_slots"][0]]: 1.0 }
#         else:
#             slot_prob = {}
#             print(f"{slot_id}: {slot_val}")
#             print(slots.keys())
#             feed_slot_probs = [slots[fs]["prob"] for fs in slot_val["feed_slots"]]
#             print(f"feed_slot_probs: {feed_slot_probs}")
#             for tid, feed_prob in feed_slot_probs[0].items():
#                 total_t_prob = 0
#                 for opp, opp_prob in feed_slot_probs[1].items():
#                     total_t_prob += feed_prob * opp_prob * get_win_prob(None, tid, opp)[0]
#                 slot_prob[tid] = total_t_prob
#             for tid, feed_prob in feed_slot_probs[1].items():
#                 total_t_prob = 0
#                 for opp, opp_prob in feed_slot_probs[0].items():
#                     total_t_prob += feed_prob * opp_prob * get_win_prob(None, tid, opp)[0]
#                 slot_prob[tid] = total_t_prob
#             slot_val["prob"] = slot_prob
#     return slots

    
# seed_teams = {}
# for index, row in seeds_df[seeds_df["Season"] == 2023].iterrows():
#     seed = row["Seed"]
#     team_id = row["TeamID"]
#     seed_teams[seed] = team_id
# print(seed_teams)
# slot_probs = get_slot_probabilities(seed_teams)
# slot_probs

In [27]:
# Load clf and scaler 
from typing import List
import pandas as pd
import pickle
with open("../Models/clf.pkl", "rb") as f:
    clf = pickle.load(f)
with open("../Models/scaler.pkl", "rb") as f:
    clf_scaler = pickle.load(f)

df = pd.read_csv("../TeamSeasons.csv")

FEATURES = [
    'AdjOE_mean', 'EFG%_mean', 'FGA3_mean', 'TO_mean', 'OR_mean', 'FT%_mean', # Offense
    'AdjDE_mean', 'OppEFG%_mean', 'OppFGA3_mean', 'OppTO_mean', 'OppOR_mean', # Defense
    'AdjNE_mean', 'Poss_mean', 'SOS', 'Q1_WinPct', 'Q2_WinPct' # Other
]

def get_diff(df:pd.DataFrame, t1_id:int, t2_id:int, season:int, features:List[str]):
    t1 = df[(df['TeamID'] == t1_id) & (df['Season'] == season)][features].fillna(0)
    t2 = df[(df['TeamID'] == t2_id) & (df['Season'] == season)][features].fillna(0)
    diff = t1.values - t2.values
    return diff[0]

def get_model_prob(model_id, season, team1, team2):
    if model_id == "clf":
        diff_v1 = get_diff(df, team1, team2, season, FEATURES)
        diff_v2 = get_diff(df, team2, team1, season, FEATURES)
        diffs = clf_scaler.transform([diff_v1, diff_v2])
        prob = clf.predict_proba(diffs)
        avg_win_prob = (prob[0][1] + prob[1][0]) / 2
        avg_loss_prob = (prob[0][0] + prob[1][1]) / 2
        return [avg_loss_prob, avg_win_prob]
    else:    
        return [0.5, 0.5]

In [31]:
p1 = (get_model_prob('clf', 2023, 1242, 1113)[1])
p2 = (get_model_prob('clf', 2023, 1113, 1242)[1])
sum([p1, p2])

1.0

In [33]:
# Build out json view of the tournament from the dataframes
for season in range(2013, 2024):
    print(f"Season: {season}")
    seasion_regions = seasons_df[seasons_df["Season"] == season]
    season_slots = slots_df[slots_df["Season"] == season]
    season_seeds = seeds_df[seeds_df["Season"] == season]
    season_results = results_df[results_df["Season"] == season]
    season_teams = season_seeds["TeamID"].unique()
    season_teams.sort()

    # Build out json view of the tournament from the dataframes
    # Use slots to build out the tournament structure
    # Each slot has a strong seed and a weak seed
    tournament = {
        "regions": {
            "W": seasion_regions["RegionW"].values[0],
            "X": seasion_regions["RegionX"].values[0],
            "Y": seasion_regions["RegionY"].values[0],
            "Z": seasion_regions["RegionZ"].values[0]
        },
        "slots": {},
        "teams": { int(team_id): get_team_data(int(team_id), season) for team_id in season_teams }
    }
    for index, row in season_slots.iterrows():
        slot = row["Slot"]
        strong_seed = row["StrongSeed"]
        weak_seed = row["WeakSeed"]
        seed_round_slots = seed_round_slots_df[seed_round_slots_df["GameSlot"] == slot]
        if slot not in tournament["slots"]:
            tournament["slots"][slot] = {}
        tournament["slots"][slot]["round_start"] = seed_round_slots["EarlyDayNum"].values[0]
        tournament["slots"][slot]["strong_seed"] = strong_seed
        tournament["slots"][slot]["weak_seed"] = weak_seed
    
    # Fill Team Seeds Mapping
    seed_teams = {}
    for index, row in season_seeds.iterrows():
        seed = row["Seed"]
        team_id = row["TeamID"]
        seed_teams[seed] = team_id

    # Fill in the first round slot
    # strong seed and week seed data from seeds data
    slot_winners, slot_probabilities = {}, {}
    slot_keys_sorted_by_day_range = sorted(tournament["slots"].keys(), key=lambda x: tournament["slots"][x]["round_start"])
    for slot in slot_keys_sorted_by_day_range:
        try:
            strong_seed = tournament["slots"][slot]["strong_seed"]
            if strong_seed.startswith("R") or strong_seed in slot_winners:
                # Fetch game winner
                strong_seed_team_id = slot_winners[strong_seed]
                strong_seed_probs = slot_probabilities[strong_seed]
            else:
                # Fetch from seeding data
                strong_seed_team_id = seed_teams[strong_seed]
                strong_seed_probs = { strong_seed_team_id: 1.0 }

            weak_seed = tournament["slots"][slot]["weak_seed"]
            if weak_seed.startswith("R") or weak_seed in slot_winners:
                weak_seed_team_id = slot_winners[weak_seed]
                weak_seed_probs = slot_probabilities[weak_seed]
            else:
                weak_seed_team_id = seed_teams[weak_seed]
                weak_seed_probs = { weak_seed_team_id: 1.0 }

            tournament["slots"][slot]["strong_seed"] = strong_seed_team_id
            tournament["slots"][slot]["weak_seed"] = weak_seed_team_id
            # Fetch slot winner from seaon_results
            if season == 2021 and slot == "R1X7":
                # VCU vs Oregon forfeit
                slot_winner, wscore, lscore = 1332, 2, 0
            else:
                game_row = season_results[season_results["WTeamID"].isin([strong_seed_team_id, weak_seed_team_id]) & season_results["LTeamID"].isin([strong_seed_team_id, weak_seed_team_id])]
                slot_winner = game_row["WTeamID"].values[0]
                wscore, lscore = game_row["WScore"].values[0], game_row["LScore"].values[0]
            tournament["slots"][slot]["winner"] = slot_winner
            tournament["slots"][slot]["wscore"] = wscore
            tournament["slots"][slot]["lscore"] = lscore

            # ---- Slot Probabilities ----
            slot_probs = {}
            for tid, team_prob in strong_seed_probs.items():
                t_prob = 0
                for opp, opp_prob in weak_seed_probs.items():
                    t_prob += team_prob * opp_prob * get_model_prob('clf', season, tid, opp)[1] # 0 index -> P(Opp Win), 1 index -> P(Team Win)
                slot_probs[tid] = t_prob
            for tid, team_prob in weak_seed_probs.items():
                t_prob = 0
                for opp, opp_prob in strong_seed_probs.items():
                    t_prob += team_prob * opp_prob * get_model_prob('clf', season, tid, opp)[1]
                slot_probs[tid] = t_prob
            slot_prob_total = sum(slot_probs.values())
            assert round(slot_prob_total, 1) == 1.0, f"Slot {slot} Probabilities do not sum to 1.0: {slot_prob_total}"
            tournament["slots"][slot]["prob"] = { t:round(v,4) for t,v in slot_probs.items() }
            slot_probabilities[slot] = slot_probs

            # print(f"{slot} winner: {slot_winner} [{game_row['WScore'].values[0]}-{game_row['LScore'].values[0]}]")
            slot_winners[slot] = slot_winner
        except Exception as e:
            print(f"Error {e}: {slot} {strong_seed} {weak_seed}, {slot_winners}")
            raise

    # Add probabilities for all matchups
    tournament['predictions'] = {}
    tourney_team_ids = list(tournament['teams'].keys())
    for i, t1_id in enumerate(tourney_team_ids):
        for t2_id in tourney_team_ids[i+1:]:
            win_probs = get_model_prob('clf', season, t1_id, t2_id)
            # IMPORTANT: 0 -> t2 wins, 1 -> t1 wins
            winner_prob = max([(wt, wp) for wt, wp in zip([t2_id, t1_id], win_probs)], key=lambda x: x[1])
            matchup_key = "_".join([str(i) for i in sorted([t1_id, t2_id])])
            # Value: [win_prob, win_team_id]
            tournament['predictions'][matchup_key] = list(winner_prob)
    with open(f"../data/web/tourney_v2/{season}.json", "w") as f:
        json.dump(tournament, f, cls=NpEncoder)

Season: 2013
Season: 2014
Season: 2015
Season: 2016
Season: 2017
Season: 2018
Season: 2019
Season: 2020
Season: 2021
Season: 2022
Season: 2023


In [96]:

# for index, row in season_slots.iterrows():
#     slot = row["Slot"]
#     strong_seed = row["StrongSeed"]
#     weak_seed = row["WeakSeed"]
#     seed_round_slots = seed_round_slots_df[seed_round_slots_df["GameSlot"] == slot]
#     if slot not in tournament["slots"]:
#         tournament["slots"][slot] = {}
#     tournament["slots"][slot]["round_start"] = seed_round_slots["EarlyDayNum"].values[0]
#     tournament["slots"][slot]["strong_seed"] = strong_seed
#     tournament["slots"][slot]["weak_seed"] = weak_seed

season_slots[['Slot', 'StrongSeed', 'WeakSeed']]
seasons_slots_2024 = pd.DataFrame(
    columns=["Slot", "StrongSeed", "WeakSeed"],
    data=[
        ["R1W1", "W01", "W16"],
        ["R1W2", "W02", "W15"],
        ["R1W3", "W03", "W14"],
        ["R1W4", "W04", "W13"],
        ["R1W5", "W05", "W12"],
        ["R1W6", "W06", "W11"],
        ["R1W7", "W07", "W10"],
        ["R1W8", "W08", "W09"],
        ["X16", "X16a", "X16b"],
        ["R1X1", "X01", "X16"],
        ["R1X2", "X02", "X15"],
        ["R1X3", "X03", "X14"],
        ["R1X4", "X04", "X13"],
        ["R1X5", "X05", "X12"],
        ["R1X6", "X06", "X11"],
        ["R1X7", "X07", "X10"],
        ["R1X8", "X08", "X09"],
        ["R1Y1", "Y01", "Y16"],
        ["R1Y2", "Y02", "Y15"],
        ["R1Y3", "Y03", "Y14"],
        ["R1Y4", "Y04", "Y13"],
        ["R1Y5", "Y05", "Y12"],
        ["R1Y6", "Y06", "Y11"],
        ["Y10", "Y10a", "Y10b"],
        ["R1Y7", "Y07", "Y10"],
        ["R1Y8", "Y08", "Y09"],
        ["Z16", "Z16a", "Z16b"],
        ["R1Z1", "Z01", "Z16"],
        ["R1Z2", "Z02", "Z15"],
        ["R1Z3", "Z03", "Z14"],
        ["R1Z4", "Z04", "Z13"],
        ["R1Z5", "Z05", "Z12"],
        ["R1Z6", "Z06", "Z11"],
        ["Z10", "Z10a", "Z10b"],
        ["R1Z7", "Z07", "Z10"],
        ["R1Z8", "Z08", "Z09"],
        ["R2W1", "R1W1", "R1W8"],
        ["R2W2", "R1W2", "R1W7"],
        ["R2W3", "R1W3", "R1W6"],
        ["R2W4", "R1W4", "R1W5"],
        ["R3W1", "R2W1", "R2W4"],
        ["R3W2", "R2W2", "R2W3"],
        ["R4W1", "R3W1", "R3W2"],
        ["R2X1", "R1X1", "R1X8"],
        ["R2X2", "R1X2", "R1X7"],
        ["R2X3", "R1X3", "R1X6"],
        ["R2X4", "R1X4", "R1X5"],
        ["R3X1", "R2X1", "R2X4"],
        ["R3X2", "R2X2", "R2X3"],
        ["R4X1", "R3X1", "R3X2"],
        ["R2Y1", "R1Y1", "R1Y8"],
        ["R2Y2", "R1Y2", "R1Y7"],
        ["R2Y3", "R1Y3", "R1Y6"],
        ["R2Y4", "R1Y4", "R1Y5"],
        ["R3Y1", "R2Y1", "R2Y4"],
        ["R3Y2", "R2Y2", "R2Y3"],
        ["R4Y1", "R3Y1", "R3Y2"],
        ["R2Z1", "R1Z1", "R1Z8"],
        ["R2Z2", "R1Z2", "R1Z7"],
        ["R2Z3", "R1Z3", "R1Z6"],
        ["R2Z4", "R1Z4", "R1Z5"],
        ["R3Z1", "R2Z1", "R2Z4"],
        ["R3Z2", "R2Z2", "R2Z3"],
        ["R4Z1", "R3Z1", "R3Z2"],
        ["R5WX", "R4W1", "R4X1"],
        ["R5YZ", "R4Y1", "R4Z1"],
        ["R6CH", "R5WX", "R5YZ"]
    ])

In [92]:

seed_round_slots_df = pd.concat([
    seed_round_slots_df,
    pd.DataFrame(columns=["Seed","GameRound","GameSlot", "EarlyDayNum","LateDayNum"],
                 data=[
                    ["X16a", 0, "X16", 134, 135],
                    ["X16b", 0, "X16", 134, 135],
                    ["Y10a", 0, "Y10", 134, 135],
                    ["Y10b", 0, "Y10", 134, 135],
                    ["Z16a", 0, "Z16", 134, 135],
                    ["Z16b", 0, "Z16", 134, 135],
                    ["Z10a", 0, "Z10", 134, 135],
                    ["Z10b", 0, "Z10", 134, 135],
                 ])])
seed_round_slots_df[seed_round_slots_df["GameSlot"].str.len() == 3]

,Seed,GameRound,GameSlot,EarlyDayNum,LateDayNum
66,W11a,0,W11,134,135
73,W11b,0,W11,134,135
86,W12a,0,W12,134,135
93,W12b,0,W12,134,135
106,W13a,0,W13,134,135
113,W13b,0,W13,134,135
126,W14a,0,W14,134,135
133,W14b,0,W14,134,135
146,W15a,0,W15,134,135
153,W15b,0,W15,134,135


In [98]:
# seed_round_slots_df[seed_round_slots_df["GameSlot"]]
seed_round_slots_df[seed_round_slots_df["Seed"] == "X16"].groupby("GameSlot").value_counts()

GameSlot  Seed  GameRound  EarlyDayNum  LateDayNum
R1X1      X16   1          136          137           1
R2X1      X16   2          138          139           1
R3X1      X16   3          143          144           1
R4X1      X16   4          145          146           1
R5WX      X16   5          152          152           1
R6CH      X16   6          154          154           1
dtype: int64

In [105]:
# Manually build the 2024 tournament
PLAY_IN_SLOTS = ["X16", "Y10", "Z16", "Z10"]
seeds_2024 = {
    # W: East
    "W01": 1163, "W16": 1391, "W08": 1194, "W09": 1321, "W05": 1361, "W12": 1412,
    "W04": 1120, "W13": 1463, "W06": 1140, "W11": 1182, "W03": 1228, "W14": 1287,
    "W07": 1450, "W10": 1179, "W02": 1235, "W15": 1355,
    # X: West
    "X01": 1314, "X16a": 1224, "X16b": 1447, "X08": 1280, "X09": 1277, "X05": 1388, "X12": 1213,
    "X04": 1104, "X13": 1158, "X06": 1155, "X11": 1307, "X03": 1124, "X14": 1159,
    "X07": 1173, "X10": 1305, "X02": 1112, "X15": 1253,
    # Y: South
    # TODO: FINISH
    "Y01": 1222, "Y16": 1255, "Y08": 1304, "Y09": 1401, "Y05": 1458, "Y12": 1241,
    "Y04": 1181, "Y13": 1436, "Y06": 1403, "Y11": 1301, "Y03": 1246, "Y14": 1324,
    "Y07": 1196, "Y10a": 1129, "Y10b": 1160, "Y02": 1266, "Y15": 1443,
    # Z: Midwest
    "Z01": 1345, "Z16a": 1286, "Z16b": 1212, "Z08": 1429, "Z09": 1395, "Z05": 1211, "Z12": 1270,
    "Z04": 1242, "Z13": 1359, "Z06": 1376, "Z11": 1332, "Z03": 1166, "Z14": 1103,
    "Z07": 1400, "Z10a": 1438, "Z10b": 1161, "Z02": 1397, "Z15": 1389
}
import re
def strip_seed(s):
    return int(re.sub("[^0-9^.]", "",s).lstrip('0'))
team_seeds_2024 = { v:strip_seed(k) for k,v in seeds_2024.items() }

seasion_regions = seasons_df[seasons_df["Season"] == 2024]
season_seeds = seeds_df[seeds_df["Season"] == 2024]
season_results = results_df[results_df["Season"] == 2024]
season_teams_2024 = list(seeds_2024.values())
season_teams_2024.sort()
season_team_data_2024 = {}
for team_id in season_teams_2024:
    season_team_data_2024[team_id] = get_team_data(team_id, 2024, team_seeds_2024)
    
tournament_2024 = {
    "regions": {
        "W": "East",
        "X": "West",
        "Y": "South",
        "Z": "Midwest"
    },
    "slots": {},
    "teams": season_team_data_2024
}


for index, row in seasons_slots_2024.iterrows():
    slot = row["Slot"]
    strong_seed = row["StrongSeed"]
    weak_seed = row["WeakSeed"]
    seed_round_slots = seed_round_slots_df[seed_round_slots_df["GameSlot"] == slot]
    if slot not in tournament_2024["slots"]:
        tournament_2024["slots"][slot] = {}
    tournament_2024["slots"][slot]["round_start"] = seed_round_slots["EarlyDayNum"].values[0]
    tournament_2024["slots"][slot]["strong_seed"] = strong_seed
    tournament_2024["slots"][slot]["weak_seed"] = weak_seed


# Fill in the first round slot
# strong seed and week seed data from seeds data
playin_proj_winners = {}
slot_winners, slot_probabilities = {}, {}
slot_keys_sorted_by_day_range = sorted(tournament_2024["slots"].keys(), key=lambda x: tournament_2024["slots"][x]["round_start"])
for slot in slot_keys_sorted_by_day_range:
    try:
        strong_seed = tournament_2024["slots"][slot]["strong_seed"]
        if strong_seed.startswith("R") or strong_seed in slot_winners:
            # Fetch game winner
            strong_seed_team_id = slot_winners[strong_seed]
            strong_seed_probs = slot_probabilities[strong_seed]
        else:
            # Fetch from seeding data
            strong_seed_team_id = seeds_2024[strong_seed]
            strong_seed_probs = { strong_seed_team_id: 1.0 }

        weak_seed = tournament_2024["slots"][slot]["weak_seed"]
        is_playin = False
        if weak_seed in PLAY_IN_SLOTS:
            is_playin = True
            print(f"FOUND A PLAYIN SLOT: {weak_seed}")
            weak_seed_probs = playin_proj_winners[weak_seed]
            weak_seed_team_id = "/".join([str(i) for i in weak_seed_probs.keys()])
            print(f"Playin Probs: {weak_seed_probs} [{weak_seed_team_id}][{slot}]")
        elif weak_seed.startswith("R") or weak_seed in slot_winners:
            weak_seed_team_id = slot_winners[weak_seed]
            weak_seed_probs = slot_probabilities[weak_seed]
        else:
            weak_seed_team_id = seeds_2024[weak_seed]
            weak_seed_probs = { weak_seed_team_id: 1.0 }

        tournament_2024["slots"][slot]["strong_seed"] = strong_seed_team_id
        tournament_2024["slots"][slot]["weak_seed"] = weak_seed_team_id
        # Not for current year
        # Fetch slot winner from seaon_results
        # if season == 2021 and slot == "R1X7":
        #     # VCU vs Oregon forfeit
        #     slot_winner, wscore, lscore = 1332, 2, 0
        # else:
        #     game_row = season_results[season_results["WTeamID"].isin([strong_seed_team_id, weak_seed_team_id]) & season_results["LTeamID"].isin([strong_seed_team_id, weak_seed_team_id])]
        #     slot_winner = game_row["WTeamID"].values[0]
        #     wscore, lscore = game_row["WScore"].values[0], game_row["LScore"].values[0]
        # tournament_2024["slots"][slot]["winner"] = slot_winner
        # tournament_2024["slots"][slot]["wscore"] = wscore
        # tournament_2024["slots"][slot]["lscore"] = lscore

        # ---- Slot Probabilities ----
        slot_probs = {}
        for tid, team_prob in strong_seed_probs.items():
            t_prob = 0
            for opp, opp_prob in weak_seed_probs.items():
                t_prob += team_prob * opp_prob * get_model_prob('clf', season, tid, opp)[1] # 0 index -> P(Opp Win), 1 index -> P(Team Win)
            slot_probs[tid] = t_prob
        for tid, team_prob in weak_seed_probs.items():
            t_prob = 0
            for opp, opp_prob in strong_seed_probs.items():
                t_prob += team_prob * opp_prob * get_model_prob('clf', season, tid, opp)[1]
            slot_probs[tid] = t_prob
        if is_playin:
            print(f"{slot}: {slot_probs}")
        slot_prob_total = sum(slot_probs.values())
        assert round(slot_prob_total, 1) == 1.0, f"Slot {slot} Probabilities do not sum to 1.0: {slot_prob_total}"
        tournament_2024["slots"][slot]["prob"] = { t:round(v,4) for t,v in slot_probs.items() }
        slot_probabilities[slot] = slot_probs
        if slot in PLAY_IN_SLOTS:
            playin_proj_winners[slot] = slot_probs

        # print(f"{slot} winner: {slot_winner} [{game_row['WScore'].values[0]}-{game_row['LScore'].values[0]}]")
        slot_winners[slot] = slot_winner
    except Exception as e:
        print(f"Error {e}: {slot} {strong_seed} {weak_seed}, {slot_winners}")
        raise

# Add probabilities for all matchups
tournament_2024['predictions'] = {}
tourney_team_ids = list(tournament_2024['teams'].keys())
for i, t1_id in enumerate(tourney_team_ids):
    for t2_id in tourney_team_ids[i+1:]:
        win_probs = get_model_prob('clf', season, t1_id, t2_id)
        # IMPORTANT: 0 -> t2 wins, 1 -> t1 wins
        winner_prob = max([(wt, wp) for wt, wp in zip([t2_id, t1_id], win_probs)], key=lambda x: x[1])
        matchup_key = "_".join([str(i) for i in sorted([t1_id, t2_id])])
        # Value: [win_prob, win_team_id]
        tournament_2024['predictions'][matchup_key] = list(winner_prob)
with open(f"../data/web/tourney_v2/2024.json", "w") as f:
    json.dump(tournament_2024, f, cls=NpEncoder)

FOUND A PLAYIN SLOT: X16
Playin Probs: {1224: 0.64375, 1447: 0.35625000000000007} [1224/1447][R1X1]
R1X1 Playin Probs: {1314: 0.9545937500000001, 1224: 0.041843750000000006, 1447: 0.0035625000000000006}
FOUND A PLAYIN SLOT: Y10
Playin Probs: {1129: 0.9098869047619047, 1160: 0.09011309523809524} [1129/1160][R1Y7]
R1Y7 Playin Probs: {1196: 0.31504298859126983, 1129: 0.6453372872023809, 1160: 0.039619724206349204}
FOUND A PLAYIN SLOT: Z16
Playin Probs: {1286: 0.7218333333333333, 1212: 0.2781666666666667} [1286/1212][R1Z1]
R1Z1 Playin Probs: {1345: 0.9675035000000001, 1286: 0.025264166666666664, 1212: 0.0072323333333333345}
FOUND A PLAYIN SLOT: Z10
Playin Probs: {1438: 0.7640833333333333, 1161: 0.23591666666666666} [1438/1161][R1Z7]
R1Z7 Playin Probs: {1400: 0.694358763888889, 1438: 0.26360875, 1161: 0.042032486111111114}


KeyboardInterrupt: 